In [4]:
!pip install numpy

     |████████████████████████████████| 1.1 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 241 kB 22.2 MB/s eta 0:00:01
  Created wheel for biogeme: filename=biogeme-3.2.6-cp36-cp36m-linux_x86_64.whl size=3432553 sha256=070f0969c1cbce5a165d0182d56c24a929a6007f93403e8f4654283619e1482b
  Stored in directory: /home/jovyan/.cache/pip/wheels/95/11/17/cce328860a3096bd4e004142e9720c6701929732aeea04ab80
Successfully built biogeme


In [ ]:
!pip uninstall biogeme
!pip install biogeme —no-cache-dir
# !pip install biogeme

In [7]:
!pip install fsspec

     |████████████████████████████████| 98 kB 5.7 MB/s eta 0:00:011


In [5]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as msg
from biogeme.expressions import Beta, DefineVariable

In [9]:
# Read the data
df = pd.read_csv('https://raw.githubusercontent.com/bumsubp/Biogeme/main/swissmetro.dat', '\t')
database = db.Database('swissmetro', df)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

# The following statement allows you to use the names of the variable
# as Python variable.
globals().update(database.variables)

In [10]:
# Here we use the "biogeme" way for backward compatibility
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)

# Parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

MU_EXISTING = Beta('MU_EXISTING', 1, 1, None, 0)
MU_PUBLIC = Beta('MU_PUBLIC', 1, 1, None, 0)
ALPHA_EXISTING = Beta('ALPHA_EXISTING', 0.5, 0, 1, 0)
ALPHA_PUBLIC = 1 - ALPHA_EXISTING

# Definition of new variables
SM_COST = SM_CO * (GA == 0)
TRAIN_COST = TRAIN_CO * (GA == 0)

# Definition of new variables: adding columns to the database
CAR_AV_SP = DefineVariable('CAR_AV_SP', CAR_AV * (SP != 0), database)
TRAIN_AV_SP = DefineVariable('TRAIN_AV_SP', TRAIN_AV * (SP != 0), database)
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED', TRAIN_TT / 100.0, database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED', TRAIN_COST / 100, database)
SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0, database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100, database)
CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100, database)
CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100, database)

# Definition of the utility functions
V1 = ASC_TRAIN + \
     B_TIME * TRAIN_TT_SCALED + \
     B_COST * TRAIN_COST_SCALED
V2 = ASC_SM + \
     B_TIME * SM_TT_SCALED + \
     B_COST * SM_COST_SCALED
V3 = ASC_CAR + \
     B_TIME * CAR_TT_SCALED + \
     B_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3}

# Associate the availability conditions with the alternatives
av = {1: TRAIN_AV_SP,
      2: SM_AV,
      3: CAR_AV_SP}

# Definition of nests
# Nest membership parameters
alpha_existing = {1: ALPHA_EXISTING,
                  2: 0.0,
                  3: 1.0}

alpha_public = {1: ALPHA_PUBLIC,
                2: 1.0,
                3: 0.0}

nest_existing = MU_EXISTING, alpha_existing
nest_public = MU_PUBLIC, alpha_public
nests = nest_existing, nest_public

# The choice model is a cross-nested logit, with availability conditions
logprob = models.logcnl_avail(V, av, nests, CHOICE)

# Define level of verbosity
logger = msg.bioMessage()
#logger.setSilent()
#logger.setWarning()
logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '11cnl'

# Estimate the parameters
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[07:13:30] < General >   Remove 26 unused variables from the database as only 10 are used.
[07:13:31] < General >   Log likelihood (N = 6768):  -6964.663 Gradient norm:      3e+03  
[07:13:31] < General >   Log likelihood (N = 6768):  -6162.407 Gradient norm:      2e+03  
[07:13:32] < General >   Log likelihood (N = 6768):  -5479.356 Gradient norm:      5e+02  
[07:13:32] < General >   Log likelihood (N = 6768):  -5443.563 Gradient norm:      5e+02  
[07:13:33] < General >   Log likelihood (N = 6768):  -5324.583 Gradient norm:      3e+02  
[07:13:33] < General >   Log likelihood (N = 6768):   -5265.52 Gradient norm:      1e+02  
[07:13:34] < General >   Log likelihood (N = 6768):  -5261.926 Gradient norm:      1e+02  
[07:13:34] < General >   Log likelihood (N = 6768):  -5257.932 Gradient norm:      9e+01  
[07:13:35] < General >   Log likelihood (N = 6768):  -5254.535 Gradient norm:      9e+01  
[07:13:36] < General >   Log likelihood (N = 6768):  -5251.128 Gradient norm:      1e+02  

In [11]:
def press_statistic(y_true, y_pred, xs):
    """
    Calculation of the `Press Statistics <https://www.otexts.org/1580>`_
    """
    res = y_pred - y_true
    hat = xs.dot(np.linalg.pinv(xs))
    den = (1 - np.diagonal(hat))
    sqr = np.square(res/den)
    return sqr.sum()

def predicted_r2(y_true, y_pred, xs):
    """
    Calculation of the `Predicted R-squared <https://rpubs.com/RatherBit/102428>`_
    """
    press = press_statistic(y_true=y_true,
                            y_pred=y_pred,
                            xs=xs
    )

    sst  = np.square( y_true - y_true.mean() ).sum()
    return 1 - press / sst